# Metadata GRPC API example

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 * grpcurl
 * pygmentize
 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html) to setup Seldon Core with an ingress.

In [30]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


# Example description

Simple, two-node graph that shows how Metadata is handled in GRPC API mode.

Model used in this example is defined in [graph_metadata](https://docs.seldon.io/projects/seldon-core/en/latest/examples/graph-metadata.html) notebook.


In [31]:
%%writefile example-grpc.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: graph-metadata-grpc
spec:
  name: test-deployment
  transport: grpc
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/metadata-generic-node:0.4
          name: node-one
          env:           
          - name: MODEL_METADATA
            value: |
              ---
              name: node-one
              versions: [ generic-node/v0.4 ]
              platform: seldon
              inputs:
              - messagetype: tensor
                schema:
                  names: [one-input]
              outputs:
              - messagetype: tensor
                schema:
                  names: [one-output]
              custom:
                author: seldon-dev
        - image: seldonio/metadata-generic-node:0.4
          name: node-two
          env:
         
          - name: MODEL_METADATA
            value: |
              ---
              name: node-two
              versions: [ generic-node/v0.4 ]
              platform: seldon
              inputs:
              - messagetype: tensor
                schema:
                  names: [two-input]
              outputs:
              - messagetype: tensor
                schema:
                  names: [two-output]
              custom:
                author: seldon-dev
    graph:
      name: node-one
      type: MODEL
      children:
      - name: node-two
        type: MODEL
        children: []   
    name: example
    replicas: 1

Overwriting example-grpc.yaml


In [32]:
!kubectl apply -f  example-grpc.yaml -n seldon

seldondeployment.machinelearning.seldon.io/graph-metadata-grpc created


In [33]:
%%bash
kubectl wait sdep/graph-metadata-grpc \
  --for=condition=ready \
  --timeout=120s \
  -n seldon

seldondeployment.machinelearning.seldon.io/graph-metadata-grpc condition met


In [44]:

from tenacity import retry, stop_after_delay, wait_exponential
import grpc
from seldon_core.proto import prediction_pb2, prediction_pb2_grpc

@retry(stop=stop_after_delay(300), wait=wait_exponential(multiplier=1, min=0.5, max=5))
def check_node_one_metadata():
    channel = grpc.insecure_channel("localhost:8003")
    stub = prediction_pb2_grpc.SeldonStub(channel)

    # Prepare the ModelMetadata request
    metadata_request = prediction_pb2.SeldonModelMetadataRequest(name="node-one")

    # Call the ModelMetadata method
    response = stub.ModelMetadata(
        metadata_request,
        metadata=(("seldon", "graph-metadata-grpc"), ("namespace", "seldon"))
    )

    # Check outputs
    assert len(response.outputs) == 1
    output_schema = response.outputs[0].schema.struct_value.fields["names"].list_value.values
    assert output_schema[0].string_value == "one-output"


check_node_one_metadata()

In [45]:
@retry(stop=stop_after_delay(300), wait=wait_exponential(multiplier=1, min=0.5, max=5))
def check_node_two_metadata():
    channel = grpc.insecure_channel("localhost:8003")
    stub = prediction_pb2_grpc.SeldonStub(channel)

    metadata_request = prediction_pb2.SeldonModelMetadataRequest(name="node-two")
    response = stub.ModelMetadata(
        metadata_request,
        metadata=(("seldon", "graph-metadata-grpc"), ("namespace", "seldon"))
    )

    # Check outputs
    assert len(response.outputs) == 1
    output_schema = response.outputs[0].schema.struct_value.fields["names"].list_value.values
    assert output_schema[0].string_value == "two-output"

check_node_two_metadata()

In [47]:
@retry(stop=stop_after_delay(300), wait=wait_exponential(multiplier=1, min=0.5, max=5))
def check_graph_metadata():
    channel = grpc.insecure_channel("localhost:8003")
    stub = prediction_pb2_grpc.SeldonStub(channel)

    # GraphMetadata uses an empty request
    empty_request = prediction_pb2.google_dot_protobuf_dot_empty__pb2.Empty()
    response = stub.GraphMetadata(
        empty_request,
        metadata=(("seldon", "graph-metadata-grpc"), ("namespace", "seldon"))
    )

    # Example assertion: check models in graph metadata
    assert "node-one" in response.models
    assert "node-two" in response.models


check_graph_metadata()

In [48]:
!kubectl delete -f example-grpc.yaml -n seldon

seldondeployment.machinelearning.seldon.io "graph-metadata-grpc" deleted
